In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

In [ ]:
#Loading dataset

In [ ]:
df=pd.read_csv('All_Errors_Corrected.csv',parse_dates=['Standardized_Date'])
df

In [ ]:
# Change the value at index 3355
df.loc[3355, 'RAW WATER FLOW IN m3/h'] = 3489.25

# Change the value at index 6791
df.loc[6791, 'RAW WATER FLOW IN m3/h'] = 2411.58


In [ ]:
def raw_water_outlier_ffill(data, column_name1, column_name2, column_name3, threshold=4000):
 
    # Adjusting values in column_name1 based on the threshold
    for i in range(1, len(data)):
        if data.loc[i, column_name1] >= threshold:
            data.loc[i, column_name1] = data.loc[i, column_name1] * 0.1
            data.loc[i, 'TREATED WATER PRODUCTION IN m3/h'] = (data.loc[i, column_name1] * 0.967).round(2)
    
    # Forward fill for column_name2 if value > 5
    for i in range(1, len(data)):
        if data.loc[i, column_name2] > 5:
            data.loc[i, column_name2] = data.loc[i - 1, column_name2]
    
    # Adjusting values in column_name3
    for i in range(1, len(data)):
        if data.loc[i, column_name3] > threshold:
            # For specific indices, perform forward fill
            if i in [6976,1757,21314, 21315, 21316]:
                data.loc[i, column_name3] = data.loc[i - 1, column_name3]
            else:
                # Check the number of digits
                value_str = str(int(data.loc[i, column_name3]))  # Convert to string for digit length check
                if len(value_str) == 6:
                    data.loc[i, column_name3] = data.loc[i, column_name3] * 0.01
                elif len(value_str) == 5:
                    data.loc[i, column_name3] = data.loc[i, column_name3] * 0.1
                    
    return data


In [ ]:
def raw_water_outlier_ffill(data, column_name1,column_name2,column_name3, threshold=4000):
 
    for i in range(1, len(data)):
        # If the value is greater than the threshold
        if data.loc[i, column_name1] >= threshold:
            # Forward fill using the previous value
            data.loc[i, column_name1] = data.loc[i, column_name1] * 0.1
            
            data.loc[i,'TREATED WATER PRODUCTION IN m3/h'] = (data.loc[i, column_name1]*0.967).round(2)
    for i in range(1, len(data)):
        # If the value is greater than the threshold
        if data.loc[i, column_name2] > 5:
            # Forward fill using the previous value
            data.loc[i, column_name2] = data.loc[i-1, column_name2]
            
    for i in range(1, len(data)):
        # If the value is greater than the threshold
        if data.loc[i, column_name3] > threshold:
            
            # Forward fill using the previous value
            data.loc[i, column_name3] = data.loc[i-1, column_name3]
    return data


In [ ]:
df = raw_water_outlier_ffill(df, 'RAW WATER FLOW IN m3/h','CLEAR WATER SUMP LEVEL IN Meter','CLEAR WATER PUMPING FLOW m3/h')
df

In [ ]:
df[11:15]

In [ ]:
columns_of_interest = [
    'RAW WATER FLOW IN m3/h',
    'CLEAR WATER SUMP LEVEL IN Meter',
    'CLEAR WATER PUMPING FLOW m3/h',
    'TREATED WATER PRODUCTION IN m3/h'
]

for col in columns_of_interest:
    mean = df[col].mean()
    std_dev = df[col].std()
    
    # Calculate lower and upper bounds using Z-score method
    lower_limit = mean - 3 * std_dev  
    upper_limit = mean + 3 * std_dev 
    
    # Identify outliers in the column
    outliers_before = df[(df[col] > upper_limit)][col]
    
    # Print column and bounds
    print(f"Column: {col}")
    print(f"Lower Limit: {lower_limit}")
    print(f"Upper Limit: {upper_limit}")
    print(f"No. of outliers: {len(outliers_before)}")
    print(outliers_before)
    print("\n")
    
    # Find above and below index values for each outlier
    for index in outliers_before.index:
        above_index = index - 1 if index > 0 else None
        below_index = index + 1 if index < len(df) - 1 else None

        # Print outlier index and value
        print(f"Outlier index: {index}, Value: {df.at[index, col]}")

        # Print above index and value if it exists
        if above_index is not None:
            print(f"Above index: {above_index}, Value: {df.at[above_index, col]}")
        else:
            print("No row above this outlier (first row).")

        # Print below index and value if it exists
        if below_index is not None:
            print(f"Below index: {below_index}, Value: {df.at[below_index, col]}")
        else:
            print("No row below this outlier (last row).")

        print("\n")


In [ ]:
max_values_df = df['CLEAR WATER PUMPING FLOW m3/h'].sort_values(ascending=False).to_frame()
max_values_df[:60]#.to_excel('clear_water_pumping_flow_outliers.xlsx')

In [ ]:
df[21313:21317]